<a href="https://colab.research.google.com/github/datascisteven/greenflash_llm_project/blob/main/Greenflash_Toxicity_Multi_Class_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# prompt: authenticate with hugging face
from huggingface_hub import login

login()

                                                                                                                                                
# Importing Necessary Libraries



In [2]:
!pip install -q datasets transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import (
    TensorDataset,
    DataLoader,
    RandomSampler,
    SequentialSampler,
    random_split,
)
from transformers import (
    BertTokenizer,
    BertModel,
    AdamW,
    get_linear_schedule_with_warmup,
)
import time
import datetime
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    roc_auc_score
)

import re
import warnings
warnings.filterwarnings("ignore")

from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 200)
# pd.set_option('display.max_colwidth', None)

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Projects/Greenflash/Toxicity/jigsaw_train_revised.csv', index_col=0)
dataset.head()

,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,race,gender,sexual_orientation,religion,ability
0,1083994,He got his money... now he lies in wait till a...,0,0,0,0,0,0,0,0,0,0,0,0
1,650904,Mad dog will surely put the liberals in mental...,1,0,0,0,0,1,0,0,0,0,0,0
2,5902188,And Trump continues his lifelong cowardice by ...,1,0,0,0,0,1,0,0,0,0,0,0
4,5410943,Tucker and Paul are both total bad ass mofo's.,1,0,0,0,0,0,0,0,0,0,0,0
5,6290444,"Cry me a river, why don't you.\nDrinking, drug...",0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
len(dataset)

1804812

In [ ]:
dataset.toxicity.value_counts()

,count
toxicity,
0,1698374
1,106438


In [ ]:
non_toxic = dataset.loc[dataset.toxicity==0]
toxic = dataset.loc[dataset.toxicity==1]

non_toxic = non_toxic.sample(n=len(toxic), random_state=42)

dataset = pd.concat([non_toxic, toxic])
dataset.toxicity.value_counts()

,count
toxicity,
0,106438
1,106438


In [ ]:
# Filter rows where the values in 'mixed_column' are not strings
non_string_rows = dataset[~dataset['comment_text'].apply(lambda x: isinstance(x, str))]

print(non_string_rows)

Empty DataFrame
Columns: [id, comment_text, toxicity, severe_toxicity, obscene, sexual_explicit, identity_attack, insult, threat, race, gender, sexual_orientation, religion, ability]
Index: []


# Preparing Environment

In [ ]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    device = torch.device("cuda")
    print("Using GPU: ", torch.cuda.current_device())
else:
    device = torch.device("cpu")

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

Using GPU:  0


# Preparing Dataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

def get_dataset(df, tokenizer, mode="train"):
    sentences, labels = df["comment_text"], df.iloc[:, 2:].to_numpy()
    max_length = 300
    in_T = []
    in_T_attn_masks = []
    for sentence in sentences:
        enc_sent_dict = tokenizer.encode_plus(
            sentence[:300],
            max_length=max_length,
            add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            truncation=True,
            return_tensors="pt",
        )
        in_T.append(enc_sent_dict["input_ids"])
        in_T_attn_masks.append(enc_sent_dict["attention_mask"])

    in_T = torch.cat(in_T, dim=0)
    in_T_attn_masks = torch.cat(in_T_attn_masks, dim=0)
    labels = torch.tensor(labels, dtype=torch.float32)
    print("Text Input: ", in_T.shape)
    print("Text Input Attention: ", in_T_attn_masks.shape)
    print("Labels: ", labels.shape)

    dataset = TensorDataset(in_T, in_T_attn_masks, labels)

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset


train_dataset, val_dataset = get_dataset(
    dataset, tokenizer=tokenizer, mode="train"
)

batch_size = 8
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset)
)
val_dataloader = DataLoader(
    val_dataset, batch_size=batch_size, sampler=SequentialSampler(val_dataset)
)

print("Data Ready!!")

Text Input:  torch.Size([212876, 300])
Text Input Attention:  torch.Size([212876, 300])
Labels:  torch.Size([212876, 12])
Data Ready!!


# Preparing Model

In [ ]:
class MultiTaskClassifier(nn.Module):
    def __init__(self, hidden_dim, num_labels):
        super(MultiTaskClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_labels = num_labels

        self.bertmodel = BertModel.from_pretrained("bert-base-uncased")
        self.ffn1 = nn.Linear(768, hidden_dim)
        self.dp1 = nn.Dropout()
        self.ffn2 = nn.Linear(hidden_dim, num_labels)

    def forward(self, in_T, in_T_attn_masks):
        hidden_states = self.bertmodel(in_T, in_T_attn_masks)[0]  # Get the first output (hidden states)
        x = torch.mean(hidden_states, dim=1)
        x = F.relu(self.ffn1(x))
        x = self.dp1(x)
        x = torch.sigmoid(self.ffn2(x))
        return x


model = MultiTaskClassifier(100, 12).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
criterion = nn.BCELoss()

# Training and Validation

In [ ]:
len(train_dataloader)

180482

In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


# def find_best_thresholds(true_labels, pred_logits):
#     best_thresholds = []
#     for i in range(pred_logits.shape[1]):
#         thresholds = np.arange(0.1, 0.9, 0.05)
#         best_f1 = 0
#         best_threshold = 0.25  # Default
#         for t in thresholds:
#             pred_labels = (pred_logits[:, i] >= t).astype(int)
#             f1 = f1_score(true_labels[:, i], pred_labels, average='macro')
#             if f1 > best_f1:
#                 best_f1 = f1
#                 best_threshold = t
#         best_thresholds.append(best_threshold)
#     return best_thresholds

In [ ]:
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

training_stats = []
total_t0 = time.time()

best_val_loss = 1e8

true_labels = torch.cat([batch[2] for batch in val_dataloader]).numpy()

for epoch in range(0, epochs):

    print(f"\n======== Epoch {epoch + 1} / {epochs} ========")
    print("Training...")

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f" Batch {step:>5,}  of  {len(train_dataloader):>5,}. Elapsed: {elapsed}. Loss: {total_train_loss / step:.5f}")

        b_in_T = batch[0].to(device)
        b_in_T_attn_masks = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        logits = model(b_in_T, b_in_T_attn_masks)
        loss = criterion(logits, b_labels)

        total_train_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)

    print(f"\n  Average training loss: {avg_train_loss:.2f}")
    print(f"  Training epoch took: {training_time}")

    print("\nRunning Validation...")

    t0 = time.time()
    model.eval()

    total_eval_loss = 0
    pred_labels = np.empty((0, 12))

    for batch in val_dataloader:
        b_in_T = batch[0].to(device)
        b_in_T_attn_masks = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            logits = model(b_in_T, b_in_T_attn_masks)
            loss = criterion(logits, b_labels)

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        # Correctly populate pred_logits with the model's output
        pred_labels = np.concatenate((pred_labels, logits), axis=0)

    avg_val_loss = total_eval_loss / len(val_dataloader)
    validation_time = format_time(time.time() - t0)

    pred_labels = np.array(
        [[int(x >= 0.25) for x in pred_labels[:, i]] for i in range(12)]
    ).transpose()

    print(f"  Validation Loss: {avg_val_loss:.2f}")
    print(f"  Validation took: {validation_time}")

    for i in range(12):
        print(f"  Accuracy (Class {i}): {accuracy_score(true_labels[:, i], pred_labels[:, i]):.2f}")
        print(f"  Macro F1-score (Class {i}): {f1_score(true_labels[:, i], pred_labels[:, i], average='macro'):.2f}")
        print(f"  Weighted F1-score (Class {i}): {f1_score(true_labels[:, i], pred_labels[:, i], average='weighted'):.2f}")
        print(f"  Confusion Matrix (Class {i}): \n{confusion_matrix(true_labels[:, i], pred_labels[:, i])}")
        print(f"  Precision (Class {i}): {precision_score(true_labels[:, i], pred_labels[:, i]):.2f}")
        print(f"  Recall (Class {i}): {recall_score(true_labels[:, i], pred_labels[:, i]):.2f}")

    training_stats.append(
        {
            "epoch": epoch + 1,
            "training_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "val_accuracy": np.mean([accuracy_score(true_labels[:, i], pred_labels[:, i]) for i in range(12)]),
            "val_macro_f1": np.mean([f1_score(true_labels[:, i], pred_labels[:, i], average="macro") for i in range(12)]),
            "val_weighted_f1": np.mean([f1_score(true_labels[:, i], pred_labels[:, i], average="weighted") for i in range(12)]),
            "val_precision": np.mean([precision_score(true_labels[:, i], pred_labels[:, i]) for i in range(12)]),
            "val_recall": np.mean([recall_score(true_labels[:, i], pred_labels[:, i]) for i in range(12)]),
            "val_accuracy_class_1": accuracy_score(true_labels[:, 0], pred_labels[:, 0]),
            "val_macro_f1_class_1": f1_score(true_labels[:, 0], pred_labels[:, 0], average="macro"),
            "val_precision_class_1": precision_score(true_labels[:, 0], pred_labels[:, 0]),
            "val_recall_class_1": recall_score(true_labels[:, 0], pred_labels[:, 0]),
            "val_accuracy_class_2": accuracy_score(true_labels[:, 1], pred_labels[:, 1]),
            "val_macro_f1_class_2": f1_score(true_labels[:, 1], pred_labels[:, 1], average="macro"),
            "val_precision_class_2": precision_score(true_labels[:, 1], pred_labels[:, 1]),
            "val_recall_class_2": recall_score(true_labels[:, 1], pred_labels[:, 1]),
            "val_accuracy_class_3": accuracy_score(true_labels[:, 2], pred_labels[:, 2]),
            "val_macro_f1_class_3": f1_score(true_labels[:, 2], pred_labels[:, 2], average="macro"),
            "val_precision_class_3": precision_score(true_labels[:, 2], pred_labels[:, 2]),
            "val_recall_class_3": recall_score(true_labels[:, 2], pred_labels[:, 2]),
            "val_accuracy_class_4": accuracy_score(true_labels[:, 3], pred_labels[:, 3]),
            "val_macro_f1_class_4": f1_score(true_labels[:, 3], pred_labels[:, 3], average="macro"),
            "val_precision_class_4": precision_score(true_labels[:, 3], pred_labels[:, 3]),
            "val_recall_class_4": recall_score(true_labels[:, 3], pred_labels[:, 3]),
            "val_accuracy_class_5": accuracy_score(true_labels[:, 4], pred_labels[:, 4]),
            "val_macro_f1_class_5": f1_score(true_labels[:, 4], pred_labels[:, 4], average="macro"),
            "val_precision_class_5": precision_score(true_labels[:, 4], pred_labels[:, 4]),
            "val_recall_class_5": recall_score(true_labels[:, 4], pred_labels[:, 4]),
            "val_accuracy_class_6": accuracy_score(true_labels[:, 5], pred_labels[:, 5]),
            "val_macro_f1_class_6": f1_score(true_labels[:, 5], pred_labels[:, 5], average="macro"),
            "val_precision_class_6": precision_score(true_labels[:, 5], pred_labels[:, 5]),
            "val_recall_class_6": recall_score(true_labels[:, 5], pred_labels[:, 5]),
            "val_accuracy_class_7": accuracy_score(true_labels[:, 6], pred_labels[:, 6]),
            "val_macro_f1_class_7": f1_score(true_labels[:, 6], pred_labels[:, 6], average="macro"),
            "val_precision_class_7": precision_score(true_labels[:, 6], pred_labels[:, 6]),
            "val_recall_class_7": recall_score(true_labels[:, 6], pred_labels[:, 6]),
            "val_accuracy_class_8": accuracy_score(true_labels[:, 7], pred_labels[:, 7]),
            "val_macro_f1_class_8": f1_score(true_labels[:, 7], pred_labels[:, 7], average="macro"),
            "val_precision_class_8": precision_score(true_labels[:, 7], pred_labels[:, 7]),
            "val_recall_class_8": recall_score(true_labels[:, 7], pred_labels[:, 7]),
            "val_accuracy_class_9": accuracy_score(true_labels[:, 8], pred_labels[:, 8]),
            "val_macro_f1_class_9": f1_score(true_labels[:, 8], pred_labels[:, 8], average="macro"),
            "val_precision_class_9": precision_score(true_labels[:, 8], pred_labels[:, 8]),
            "val_recall_class_9": recall_score(true_labels[:, 8], pred_labels[:, 8]),
            "val_accuracy_class_10": accuracy_score(true_labels[:, 9], pred_labels[:, 9]),
            "val_macro_f1_class_10": f1_score(true_labels[:, 9], pred_labels[:, 9], average="macro"),
            "val_precision_class_10": precision_score(true_labels[:, 9], pred_labels[:, 9]),
            "val_recall_class_10": recall_score(true_labels[:, 9], pred_labels[:, 9]),
            "val_accuracy_class_11": accuracy_score(true_labels[:, 10], pred_labels[:, 10]),
            "val_macro_f1_class_11": f1_score(true_labels[:, 10], pred_labels[:, 10], average="macro"),
            "val_precision_class_11": precision_score(true_labels[:, 10], pred_labels[:, 10]),
            "val_recall_class_11": recall_score(true_labels[:, 10], pred_labels[:, 10]),
            "val_accuracy_class_12": accuracy_score(true_labels[:, 11], pred_labels[:, 11]),
            "val_macro_f1_class_12": f1_score(true_labels[:, 11], pred_labels[:, 11], average="macro"),
            "val_precision_class_12": precision_score(true_labels[:, 11], pred_labels[:, 11]),
            "val_recall_class_12": recall_score(true_labels[:, 11], pred_labels[:, 11]),
            "training_time": training_time,
            "val_time": validation_time,
        }
    )

    model_path = f"toxicity_multi_model_state_dict_epoch_{epoch + 1}.pt"
    torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'avg_val_loss': avg_val_loss,
            }, model_path)

    command = "cp -r /content/toxicity_multi_model_state_dict_epoch_{0}.pt /content/drive/MyDrive/Projects/Greenflash/toxicity_multi_model_state_dict_epoch_{1}.pt".format(epoch + 1, epoch + 1)
    os.system(command)

print("")
stats_path = "toxicity_multi_training_stats_pickle"
pd.DataFrame(training_stats).to_pickle(stats_path)
!cp -r /content/toxicity_multi_training_stats_pickle /content/drive/MyDrive/Projects/Greenflash/toxicity_multi_training_stats_pickle

# save the model
model_path = "toxicity_multi_model.pth"
torch.save(model, model_path)
!cp -r /content/toxicity_multi_model.pth /content/drive/MyDrive/Projects/Greenflash/toxicity_multi_model.pth

print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))


======== Epoch 1 / 5 ========
Training...
 Batch   500  of  21,288. Elapsed: 0:01:32. Loss: 0.07896
 Batch 1,000  of  21,288. Elapsed: 0:03:04. Loss: 0.07959
 Batch 1,500  of  21,288. Elapsed: 0:04:36. Loss: 0.08232
 Batch 2,000  of  21,288. Elapsed: 0:06:08. Loss: 0.08179
 Batch 2,500  of  21,288. Elapsed: 0:07:40. Loss: 0.08076
 Batch 3,000  of  21,288. Elapsed: 0:09:12. Loss: 0.08098
 Batch 3,500  of  21,288. Elapsed: 0:10:44. Loss: 0.07996
 Batch 4,000  of  21,288. Elapsed: 0:12:16. Loss: 0.07979
 Batch 4,500  of  21,288. Elapsed: 0:13:48. Loss: 0.07983
 Batch 5,000  of  21,288. Elapsed: 0:15:20. Loss: 0.07969
 Batch 5,500  of  21,288. Elapsed: 0:16:52. Loss: 0.07958
 Batch 6,000  of  21,288. Elapsed: 0:18:24. Loss: 0.07943
 Batch 6,500  of  21,288. Elapsed: 0:19:56. Loss: 0.07930
 Batch 7,000  of  21,288. Elapsed: 0:21:28. Loss: 0.07938
 Batch 7,500  of  21,288. Elapsed: 0:23:00. Loss: 0.07937
 Batch 8,000  of  21,288. Elapsed: 0:24:31. Loss: 0.07939
 Batch 8,500  of  21,288. Ela

In [ ]:
# epochs = 5
# total_steps = len(train_dataloader) * epochs
# scheduler = get_linear_schedule_with_warmup(
#     optimizer, num_warmup_steps=0, num_training_steps=total_steps
# )

# training_stats = []
# total_t0 = time.time()

# best_val_loss = 1e8

# true_labels = torch.cat([batch[2] for batch in val_dataloader]).numpy()

# for epoch in range(0, epochs):

#     print(f"\n======== Epoch {epoch + 1} / {epochs} ========")
#     print("Training...")

#     t0 = time.time()
#     total_train_loss = 0

#     model.train()

#     for step, batch in enumerate(train_dataloader):

#         if step % 5000 == 0 and not step == 0:
#             elapsed = format_time(time.time() - t0)
#             print(f" Batch {step:>5,}  of  {len(train_dataloader):>5,}. Elapsed: {elapsed}. Loss: {total_train_loss / step:.5f}")

#         b_in_T = batch[0].to(device)
#         b_in_T_attn_masks = batch[1].to(device)
#         b_labels = batch[2].to(device)

#         model.zero_grad()

#         logits = model(b_in_T, b_in_T_attn_masks)
#         loss = criterion(logits, b_labels)

#         total_train_loss += loss.item()
#         loss.backward()

#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()
#         scheduler.step()

#     avg_train_loss = total_train_loss / len(train_dataloader)
#     training_time = format_time(time.time() - t0)

#     print(f"\n  Average training loss: {avg_train_loss:.2f}")
#     print(f"  Training epoch took: {training_time}")

#     print("\nRunning Validation...")

#     t0 = time.time()
#     model.eval()

#     total_eval_loss = 0
#     pred_labels = np.empty((0, 12))
#     pred_logits = np.empty((0, 12))

#     for batch in val_dataloader:
#         b_in_T = batch[0].to(device)
#         b_in_T_attn_masks = batch[1].to(device)
#         b_labels = batch[2].to(device)

#         with torch.no_grad():
#             logits = model(b_in_T, b_in_T_attn_masks)
#             loss = criterion(logits, b_labels)

#         total_eval_loss += loss.item()

#         logits = logits.detach().cpu().numpy()
#         # Correctly populate pred_logits with the model's output
#         pred_logits = np.concatenate((pred_logits, logits), axis=0)

#     avg_val_loss = total_eval_loss / len(val_dataloader)
#     validation_time = format_time(time.time() - t0)

#     # Find best thresholds using validation logits
#     best_thresholds = find_best_thresholds(true_labels, pred_logits)

#     # Apply the best threshold for each class
#     pred_labels = np.array(
#         [[int(pred_logits[j, i] >= best_thresholds[i]) for i in range(12)] for j in range(pred_logits.shape[0])]
#     )

#     # pred_labels = np.array(
#     #     [[int(x >= 0.25) for x in pred_labels[:, i]] for i in range(12)]
#     # ).transpose()

#     print(f"  Validation Loss: {avg_val_loss:.2f}")
#     print(f"  Validation took: {validation_time}")

#     for i in range(12):
#         print(f"  Best Threshold for Class {i}: {best_thresholds[i]:.2f}")
#         print(f"  Accuracy (Class {i}): {accuracy_score(true_labels[:, i], pred_labels[:, i]):.2f}")
#         print(f"  Macro F1-score (Class {i}): {f1_score(true_labels[:, i], pred_labels[:, i], average='macro'):.2f}")
#         print(f"  Weighted F1-score (Class {i}): {f1_score(true_labels[:, i], pred_labels[:, i], average='weighted'):.2f}")
#         print(f"  Confusion Matrix (Class {i}): \n{confusion_matrix(true_labels[:, i], pred_labels[:, i])}")
#         print(f"  Precision (Class {i}): {precision_score(true_labels[:, i], pred_labels[:, i]):.2f}")
#         print(f"  Recall (Class {i}): {recall_score(true_labels[:, i], pred_labels[:, i]):.2f}")

#     training_stats.append(
#         {
#             "epoch": epoch + 1,
#             "training_loss": avg_train_loss,
#             "val_loss": avg_val_loss,
#             "val_accuracy": np.mean([accuracy_score(true_labels[:, i], pred_labels[:, i]) for i in range(12)]),
#             "val_macro_f1": np.mean([f1_score(true_labels[:, i], pred_labels[:, i], average="macro") for i in range(12)]),
#             "val_weighted_f1": np.mean([f1_score(true_labels[:, i], pred_labels[:, i], average="weighted") for i in range(12)]),
#             "val_precision": np.mean([precision_score(true_labels[:, i], pred_labels[:, i]) for i in range(12)]),
#             "val_recall": np.mean([recall_score(true_labels[:, i], pred_labels[:, i]) for i in range(12)]),
#             "val_accuracy_class_1": accuracy_score(true_labels[:, 0], pred_labels[:, 0]),
#             "val_macro_f1_class_1": f1_score(true_labels[:, 0], pred_labels[:, 0], average="macro"),
#             "val_precision_class_1": precision_score(true_labels[:, 0], pred_labels[:, 0]),
#             "val_recall_class_1": recall_score(true_labels[:, 0], pred_labels[:, 0]),
#             "best_threshold_class_1": best_thresholds[0],
#             "val_accuracy_class_2": accuracy_score(true_labels[:, 1], pred_labels[:, 1]),
#             "val_macro_f1_class_2": f1_score(true_labels[:, 1], pred_labels[:, 1], average="macro"),
#             "val_precision_class_2": precision_score(true_labels[:, 1], pred_labels[:, 1]),
#             "val_recall_class_2": recall_score(true_labels[:, 1], pred_labels[:, 1]),
#             "best_threshold_class_2": best_thresholds[1],
#             "val_accuracy_class_3": accuracy_score(true_labels[:, 2], pred_labels[:, 2]),
#             "val_macro_f1_class_3": f1_score(true_labels[:, 2], pred_labels[:, 2], average="macro"),
#             "val_precision_class_3": precision_score(true_labels[:, 2], pred_labels[:, 2]),
#             "val_recall_class_3": recall_score(true_labels[:, 2], pred_labels[:, 2]),
#             "best_threshold_class_3": best_thresholds[2],
#             "val_accuracy_class_4": accuracy_score(true_labels[:, 3], pred_labels[:, 3]),
#             "val_macro_f1_class_4": f1_score(true_labels[:, 3], pred_labels[:, 3], average="macro"),
#             "val_precision_class_4": precision_score(true_labels[:, 3], pred_labels[:, 3]),
#             "val_recall_class_4": recall_score(true_labels[:, 3], pred_labels[:, 3]),
#             "best_threshold_class_4": best_thresholds[3],
#             "val_accuracy_class_5": accuracy_score(true_labels[:, 4], pred_labels[:, 4]),
#             "val_macro_f1_class_5": f1_score(true_labels[:, 4], pred_labels[:, 4], average="macro"),
#             "val_precision_class_5": precision_score(true_labels[:, 4], pred_labels[:, 4]),
#             "val_recall_class_5": recall_score(true_labels[:, 4], pred_labels[:, 4]),
#             "best_threshold_class_5": best_thresholds[4],
#             "val_accuracy_class_6": accuracy_score(true_labels[:, 5], pred_labels[:, 5]),
#             "val_macro_f1_class_6": f1_score(true_labels[:, 5], pred_labels[:, 5], average="macro"),
#             "val_precision_class_6": precision_score(true_labels[:, 5], pred_labels[:, 5]),
#             "val_recall_class_6": recall_score(true_labels[:, 5], pred_labels[:, 5]),
#             "best_threshold_class_6": best_thresholds[5],
#             "val_accuracy_class_7": accuracy_score(true_labels[:, 6], pred_labels[:, 6]),
#             "val_macro_f1_class_7": f1_score(true_labels[:, 6], pred_labels[:, 6], average="macro"),
#             "val_precision_class_7": precision_score(true_labels[:, 6], pred_labels[:, 6]),
#             "val_recall_class_7": recall_score(true_labels[:, 6], pred_labels[:, 6]),
#             "best_threshold_class_7": best_thresholds[6],
#             "val_accuracy_class_8": accuracy_score(true_labels[:, 7], pred_labels[:, 7]),
#             "val_macro_f1_class_8": f1_score(true_labels[:, 7], pred_labels[:, 7], average="macro"),
#             "val_precision_class_8": precision_score(true_labels[:, 7], pred_labels[:, 7]),
#             "val_recall_class_8": recall_score(true_labels[:, 7], pred_labels[:, 7]),
#             "best_threshold_class_8": best_thresholds[7],
#             "val_accuracy_class_9": accuracy_score(true_labels[:, 8], pred_labels[:, 8]),
#             "val_macro_f1_class_9": f1_score(true_labels[:, 8], pred_labels[:, 8], average="macro"),
#             "val_precision_class_9": precision_score(true_labels[:, 8], pred_labels[:, 8]),
#             "val_recall_class_9": recall_score(true_labels[:, 8], pred_labels[:, 8]),
#             "best_threshold_class_9": best_thresholds[8],
#             "val_accuracy_class_10": accuracy_score(true_labels[:, 9], pred_labels[:, 9]),
#             "val_macro_f1_class_10": f1_score(true_labels[:, 9], pred_labels[:, 9], average="macro"),
#             "val_precision_class_10": precision_score(true_labels[:, 9], pred_labels[:, 9]),
#             "val_recall_class_10": recall_score(true_labels[:, 9], pred_labels[:, 9]),
#             "best_threshold_class_10": best_thresholds[9],
#             "val_accuracy_class_11": accuracy_score(true_labels[:, 10], pred_labels[:, 10]),
#             "val_macro_f1_class_11": f1_score(true_labels[:, 10], pred_labels[:, 10], average="macro"),
#             "val_precision_class_11": precision_score(true_labels[:, 10], pred_labels[:, 10]),
#             "val_recall_class_11": recall_score(true_labels[:, 10], pred_labels[:, 10]),
#             "best_threshold_class_11": best_thresholds[10],
#             "val_accuracy_class_12": accuracy_score(true_labels[:, 11], pred_labels[:, 11]),
#             "val_macro_f1_class_12": f1_score(true_labels[:, 11], pred_labels[:, 11], average="macro"),
#             "val_precision_class_12": precision_score(true_labels[:, 11], pred_labels[:, 11]),
#             "val_recall_class_12": recall_score(true_labels[:, 11], pred_labels[:, 11]),
#             "best_threshold_class_12": best_thresholds[11],
#             "training_time": training_time,
#             "val_time": validation_time,
#         }
#     )

#     model_path = f"toxicity_multi_model_state_dict_epoch_{epoch + 1}.pt"
#     torch.save({
#             'epoch': epoch + 1,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'avg_val_loss': avg_val_loss,
#             }, model_path)

#     command = "cp -r /content/toxicity_multi_model_state_dict_epoch_{0}.pt /content/drive/MyDrive/Projects/Greenflash/toxicity_multi_model_state_dict_epoch_{1}.pt".format(epoch + 1, epoch + 1)
#     os.system(command)

# print("")
# stats_path = "toxicity_multi_training_stats_pickle"
# pd.DataFrame(training_stats).to_pickle(stats_path)
# !cp -r /content/toxicity_multi_training_stats_pickle /content/drive/MyDrive/Projects/Greenflash/toxicity_multi_training_stats_pickle

# # save the model
# model_path = "toxicity_multi_model.pth"
# torch.save(model, model_path)
# !cp -r /content/toxicity_multi_model.pth /content/drive/MyDrive/Projects/Greenflash/toxicity_multi_model.pth

# print("Training complete!")
# print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))


======== Epoch 1 / 5 ========
Training...
 Batch 5,000  of  180,482. Elapsed: 0:15:04. Loss: 0.07661
 Batch 10,000  of  180,482. Elapsed: 0:30:09. Loss: 0.05995
 Batch 15,000  of  180,482. Elapsed: 0:45:13. Loss: 0.05332
 Batch 20,000  of  180,482. Elapsed: 1:00:18. Loss: 0.04944
 Batch 25,000  of  180,482. Elapsed: 1:15:28. Loss: 0.04717
 Batch 30,000  of  180,482. Elapsed: 1:30:40. Loss: 0.04551
 Batch 35,000  of  180,482. Elapsed: 1:45:53. Loss: 0.04412
 Batch 40,000  of  180,482. Elapsed: 2:01:06. Loss: 0.04301
 Batch 45,000  of  180,482. Elapsed: 2:16:19. Loss: 0.04217
 Batch 50,000  of  180,482. Elapsed: 2:31:32. Loss: 0.04146
 Batch 55,000  of  180,482. Elapsed: 2:46:45. Loss: 0.04093
 Batch 60,000  of  180,482. Elapsed: 3:01:58. Loss: 0.04030
 Batch 65,000  of  180,482. Elapsed: 3:17:11. Loss: 0.03979
 Batch 70,000  of  180,482. Elapsed: 3:32:24. Loss: 0.03931
 Batch 75,000  of  180,482. Elapsed: 3:47:37. Loss: 0.03889
 Batch 80,000  of  180,482. Elapsed: 4:02:51. Loss: 0.0385

In [ ]:
# Load the training stats from the pickle file
stats_path = "/content/toxicity_training_stats_pickle"  # Replace this with your actual file path
training_stats = pd.read_pickle(stats_path)

# Display the loaded training stats
print(training_stats)

   epoch  training_loss  val_loss  val_accuracy  val_macro_f1  \
0      1       0.059914  0.043520      0.978683      0.822396   
1      2       0.041306  0.043623      0.983743      0.810349   
2      3       0.031902  0.047829      0.982589      0.833568   
3      4       0.024000  0.061770      0.983816      0.839387   
4      5       0.017549  0.068093      0.983383      0.835246   

   val_weighted_f1 training_time val_time  
0         0.980230       1:49:44  0:08:21  
1         0.983732       1:49:44  0:08:21  
2         0.983280       1:49:39  0:08:21  
3         0.984023       1:49:40  0:08:21  
4         0.983609       1:49:40  0:08:20  


# Inference


In [10]:
test = pd.read_csv('/content/drive/MyDrive/Projects/Greenflash/Toxicity/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv', index_col=0)
test.head()

,comment_text,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,...,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability
id,,,,,,,,,,,,,,,,,,,,,
7097320,[ Integrity means that you pay your debts.]\n\...,2017-09-13 20:12:01.484121+00,21,5945023.0,376974,approved,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7097321,This is malfeasance by the Administrator and t...,2017-05-17 07:01:51.902566+00,55,NaN,335003,approved,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7097322,@Rmiller101 - Spoken like a true elitist. But ...,2016-12-02 17:12:12.920957+00,54,649389.0,154126,approved,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7097323,"Paul: Thank you for your kind words. I do, in...",2017-04-21 14:58:05.474657+00,13,5158666.0,328376,approved,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7097324,Sorry you missed high school. Eisenhower sent ...,2017-10-01 19:43:12.373229+00,102,6061626.0,383983,approved,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
test = pd.read_csv('/content/drive/MyDrive/Projects/Greenflash/Toxicity/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv', index_col=0)
test.head()

,comment_text,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,...,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability
id,,,,,,,,,,,,,,,,,,,,,
7000000,Jeff Sessions is another one of Trump's Orwell...,2017-01-26 07:37:38.422417+00,13,NaN,164149,approved,0,0,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7000001,I actually inspected the infrastructure on Gra...,2016-12-03 20:38:21.204649+00,54,655260.0,154341,approved,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7000002,No it won't . That's just wishful thinking on ...,2017-05-05 18:07:58.560078+00,21,5219683.0,332005,approved,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7000003,Instead of wringing our hands and nibbling the...,2017-02-09 14:46:30.189044+00,13,NaN,167110,approved,0,0,1,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7000004,how many of you commenters have garbage piled ...,2016-10-15 01:18:40.521923+00,21,NaN,148484,approved,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
test_texts = pd.read_csv('/content/drive/MyDrive/Projects/Greenflash/Toxicity/Jigsaw Toxic Comment Classification Challenge/test.csv', index_col=0)
test_labels = pd.read_csv('/content/drive/MyDrive/Projects/Greenflash/Toxicity/Jigsaw Toxic Comment Classification Challenge/test_labels.csv', index_col=0)

test_df = test_texts.join(test_labels)
test_df = test_df.loc[test_df['toxic'] != -1]
test_df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0


In [8]:
len(test_df)

63978

In [ ]:
test_df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0


In [ ]:
test_df.toxic.value_counts()

,count
toxic,
0,57888
1,6090


In [ ]:
def prepare_test_data(test_df, tokenizer):
    sentences = test_df["comment_text"]
    max_length = 300
    in_T = []
    in_T_attn_masks = []

    for sentence in sentences:
        enc_sent_dict = tokenizer.encode_plus(
            sentence[:300],
            max_length=max_length,
            add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            truncation=True,
            return_tensors="pt",
        )
        in_T.append(enc_sent_dict["input_ids"])
        in_T_attn_masks.append(enc_sent_dict["attention_mask"])

    in_T = torch.cat(in_T, dim=0)
    in_T_attn_masks = torch.cat(in_T_attn_masks, dim=0)

    dataset = TensorDataset(in_T, in_T_attn_masks)
    return dataset

test_dataset = prepare_test_data(test_df, tokenizer)

test_dataloader = DataLoader(test_dataset, batch_size=8, sampler=SequentialSampler(test_dataset))

In [ ]:
# Model class must be defined somewhere
model = torch.load('/content/toxicity_model.pth', weights_only=False)
model.eval()

MultiTaskClassifier(
  (bertmodel): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
# model = MultiTaskClassifier(hidden_dim=100, num_labels=6).to(device)

# checkpoint = torch.load('/content/model_state_dict_4.pt')
# model.load_state_dict(checkpoint['model_state_dict'])

# model.eval()

predictions = []
with torch.no_grad():  # No need to track gradients for inference
    for batch in test_dataloader:
        b_in_T = batch[0].to(device)
        b_in_T_attn_masks = batch[1].to(device)

        # Forward pass
        logits = model(b_in_T, b_in_T_attn_masks)

        # Apply sigmoid to get probabilities for each of the 6 tasks
        probs = torch.sigmoid(logits)

        # Store the predictions
        predictions.append(probs.cpu().numpy())

# Convert the list of predictions into a single numpy array
predictions = np.vstack(predictions)

# Apply thresholding for each task to get binary predictions
threshold = 0.5
binary_predictions = (predictions >= threshold).astype(int)

In [ ]:
binary_predictions.shape

(63978, 6)

In [ ]:
binary_predictions

array([[1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       ...,
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1]])

In [ ]:
len(test_df)

63978

In [ ]:
# prompt: Code for confusion matrix

true_labels = test_df.iloc[:, 2:].to_numpy()
for i in range(6):
  print(f"Confusion Matrix (Class {i}): \n{confusion_matrix(true_labels[:, i], binary_predictions[:, i])}")



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def draw_confusion_matrix(true, preds):
   conf_matx = confusion_matrix(true, preds)
   sns.heatmap(
      conf_matx,
      annot=True,
      annot_kws={"size": 12},
      fmt='g',
      cbar=False,
      cmap="viridis"
   )
   plt.show()

draw_confusion_matrix()